## EyeON: Eye on Operational Networks
### a firmware inventory/threat analysis tool


In [1]:
from eyeon import observe
from pprint import pprint

### Objects
EyeON consists of two parts - an `observe` call and a `parse` call. `observe` works on a single file to return a suite of identifying metrics. `parse` calls `observe` recursively, returning an observation for each file in a directory. Both of these can be run either from a library import or a `CLI` command.

In [2]:
obs = observe.Observe("./tests/binaries/Wintap/Wintap.exe")


### Data Standard
Depending on the file type, e.g. PE or ELF, different observations will be collected. 
For instance, PE files typically contain more metadata and have signature information. Here we show some high-level characteristics, and we can dig into the certificates more thoroughly.

In [3]:
print("authentihash:", obs.authentihash)
print("filename:", obs.filename)
print("file magic:", obs.magic)
print("signature_validation:", obs.signatures[0]["verification"])

authentihash: b8ea76cd7d5bccd923fedbb75041939b0249f86f
filename: Wintap.exe
file magic: PE32 executable (GUI) Intel 80386 Mono/.Net assembly, for MS Windows
signature_validation: OK


In [4]:
pprint(obs.metadata)

{'FileInfo': {'Assembly Version': '6.1.0.4',
              'Comments': 'An extensible host based agent for Windows '
                          'providing realtime event collection, analytics and '
                          'response',
              'CompanyName': 'Lawrence Livermore National Laboratory',
              'FileDescription': 'Wintap',
              'FileVersion': '6.1.0.4',
              'InternalName': 'Wintap.exe',
              'LegalCopyright': 'Copyright © Lawrence Livermore Laboratory '
                                '2024',
              'LegalTrademarks': '',
              'OriginalFilename': 'Wintap.exe',
              'ProductName': 'Wintap',
              'ProductVersion': '6.1.0.4'},
 'OS': 'Windows',
 'dllRedirectionLocal': False,
 'dotnetAssembly': [{'Culture': '',
                     'Flags': {'DisableJitCompileOptimizer': False,
                               'EnableJitCompileTracking': False,
                               'PA_AMD64': False,
             

In [5]:
for sig in obs.signatures:
    print("digest algorithm:", sig["digest_algorithm"])
    print("digest value:", sig["sha1"])
    print("signers", sig["signers"])
    print("cert validation:", sig["verification"])
    for cert in sig["certs"]:
        pprint(cert)
        break
    break

digest algorithm: ALGORITHMS.SHA_1
digest value: b8ea76cd7d5bccd923fedbb75041939b0249f86f
signers SHA_1/RSA - C=US, O=DigiCert\, Inc., CN=DigiCert Trusted G4 Code Signing RSA4096 SHA384 2021 CA1 - 2 auth attr - 1 unauth attr
cert validation: OK
{'RSA_key_size': '4096 bits',
 'basic_constraints': 'CA=true',
 'cert._version': '3',
 'certificate_policies': 'Any Policy',
 'expires_on': '2031-11-09 23:59:59',
 'issued_on': '2022-08-01 00:00:00',
 'issuer_name': 'C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert '
                'Assured ID Root CA',
 'key_usage': 'Digital Signature, Key Cert Sign, CRL Sign',
 'serial_number': '0E:9B:18:8E:F9:D0:2D:E7:EF:DB:50:E2:08:40:18:5A',
 'sha256': '33846b545a49c9be4903c60e01713c1bd4e4ef31ea65cd95d69e62794f30b941',
 'signed_using': 'RSA with SHA-384',
 'subject_name': 'C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert '
                 'Trusted Root G4'}


There is also a Command Line component installed with the `eyeon` library containing 2 options: `eyeon observe` and `eyeon parse`.
`observe` generates output for a single file, whereas `parse` scans a directory.

It can be called as below (note `!` executes a terminal command):

In [6]:
! eyeon --help

usage: eyeon [-h] [-o OUTPUT_DIR] [-g LOG_FILE] [-v LOG_LEVEL]
             {observe,parse,checksum} ...

Eye on Operational techNology, an update tracker for OT devices

positional arguments:
  {observe,parse,checksum}
                        sub-command help
    observe             observe help
    parse               parse help
    checksum            checksum help

options:
  -h, --help            show this help message and exit
  -o OUTPUT_DIR, --output-dir OUTPUT_DIR
                        Path to results directory. Defaults to $pwd. Can set
                        on $EYEON_OUTPUT.
  -g LOG_FILE, --log-file LOG_FILE
                        Output file for log. If none, prints to console.
  -v LOG_LEVEL, --log-level LOG_LEVEL
                        Set the log level. Defaults to ERROR.


In [7]:
! eyeon --output-dir ./outputs observe ./tests/binaries/Wintap/Wintap.exe
! jq . ./outputs/Wintap.*

{
  "uuid": "c90bb985-f2f4-4da5-a20b-e13860b5c421",
  "bytecount": 201080,
  "filename": "Wintap.exe",
  "signatures": [
    {
      "certs": [
        {
          "cert._version": "3",
          "sha256": "33846b545a49c9be4903c60e01713c1bd4e4ef31ea65cd95d69e62794f30b941",
          "serial_number": "0E:9B:18:8E:F9:D0:2D:E7:EF:DB:50:E2:08:40:18:5A",
          "issuer_name": "C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert Assured ID Root CA",
          "subject_name": "C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert Trusted Root G4",
          "issued_on": "2022-08-01 00:00:00",
          "expires_on": "2031-11-09 23:59:59",
          "signed_using": "RSA with SHA-384",
          "RSA_key_size": "4096 bits",
          "basic_constraints": "CA=true",
          "key_usage": "Digital Signature, Key Cert Sign, CRL Sign",
          "certificate_policies": "Any Policy"
        },
        {
          "cert._version": "3",
          "sha256": "552f7bdcf1a7af9e6ce672017f4f12abf772